In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import haversine

## 
Assumptions:
1. Number of people served in food pantry data = number of pounds
2. Approx Capacity = "Big M"<<Total_Supply to start
3. 

In [7]:
TRACT_PATH = "data/model/tract_table.csv"
METRICS    = "data/tabular/nta_with_metrics.csv"                    # e.g., "data/supply_gap_by_neighborhood.csv" or None
NTA_PATH = "data/model/nta_table.csv"
PANTRIES_PATH = "data/model/pantries_table.csv" 

In [8]:
# If PANTRIES_PATH is CSV, specify columns:
PANTRY_LAT_COL = "lat"
PANTRY_LON_COL = "lng"

CRS_LATLON    = "EPSG:4326"
CRS_PROJECTED = "EPSG:6539"  # NYC-appropriate projected CRS

### Information We Will Need to use in our optimization model:
1. Number of pantries within each NTA (neighborhood tabulation area) - coincides with metrics of food supply gap
2. Pairwise distance between all pantries and centroids of NTAs
3. Total Supply

In [9]:
# A: Food pantries
pantries = pd.read_csv(PANTRIES_PATH)

NUM_PANTRIES = pantries.shape[0]
print(f"Number of food pantries: {NUM_PANTRIES}")

pantries.head()

Number of food pantries: 515


,FID,type_fp,type_sk,program,org_phone,distadd,distboro,distzip,dist_location_info,fp_days_orig,...,sk_fri_open3,sk_fri_close3,sk_sat_open3,sk_sat_close3,sk_sun_open3,sk_sun_close3,geometry,geoid,nta2020,id
0,1,FP,NaN,CHURCH OF ST. NICHOLAS OF TOLENTINE,(718) 295-6800,"2345 University Ave, Bronx, New York, 10468",BX,10468,BASEMENT FORDHAM RD ENTRANCE,TUE,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-73.905672642358 40.862216353818),36005025500,BX0701,0
1,2,FP,NaN,BREAD OF LIFE FOOD PANTRY,(347) 235-3723,"1104 Elder Ave, Bronx, New York, 10472",BX,10472,#15,SAT,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-73.8785391641132 40.8264247055377),36005005002,BX0901,1
2,4,FP,NaN,CHANCE FOR CHILDREN,(347) 616-3228,"11 Mc Keever Pl, Brooklyn, New York, 11225",BK,11225,1ST FLOOR (BASEMENT LEVEL),FRI,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-73.9584754469383 40.6656935813309),36047032500,BK0901,2
3,6,FP,NaN,HEALTH ESSENTIAL ASSOCIATION INC (BK),(646) 515-6898,"2101 E 16th St, Brooklyn, New York, 11229",BK,11229,2ND FLOOR,FRI (4TH),...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-73.955282193588 40.598606630507),36047058000,BK1502,3
4,10,FPHA,NaN,AIDS CENTER OF QUEENS COUNTY (WOODSIDE),(718) 472-9400,"62-07 Woodside Ave, Woodside, New York, 11377",QN,11377,3RD FLOOR,"TUE, THUR",...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-73.902187195561 40.744659030225),36081026100,QN0203,4


**Pantries Per Neighborhood**

In [10]:
metrics_by_nta = pd.read_csv(NTA_PATH)
NUM_NEIGHBORHOODS = metrics_by_nta.shape[0]
print(f"Number of neighborhoods: {NUM_NEIGHBORHOODS}")

metrics_by_nta.head()

Number of neighborhoods: 197


,nta2020,geometry,geoid,Year,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank,pantry_count
0,BK0101,POINT (998241.9574088433 205056.93764567457),36047056301,2025-01-01,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0,1
1,BK0102,POINT (995721.0164833779 199741.71509407158),36047051300,2025-01-01,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0,1
2,BK0103,POINT (996267.1178804908 195489.48591761288),36047050900,2025-01-01,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0,1
3,BK0104,POINT (1001857.1708558884 199157.09185645176),36047048900,2025-01-01,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0,1
4,BK0201,POINT (985683.5935392772 192653.28067751398),36047000301,2025-01-01,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0,0


In [11]:
# B: Supply gap by neighborhood
gap_per_nta= metrics_by_nta[['nta2020','ntaname','geoid','Supply Gap (lbs.)']]
gap_per_nta.head()

,nta2020,ntaname,geoid,Supply Gap (lbs.)
0,BK0101,Greenpoint,36047056301,"1,153,881.91747087"
1,BK0102,Williamsburg,36047051300,"1,011,421.07761282"
2,BK0103,South Williamsburg,36047050900,"2,090,950.72621075"
3,BK0104,East Williamsburg,36047048900,"1,224,484.03196005"
4,BK0201,Brooklyn Heights,36047000301,"322,651.410235809"


In [12]:
# C: Pantries per NTA:
pantries_per_nta = metrics_by_nta[['nta2020','pantry_count', 'ntaname']]
pantries_per_nta.head()

,nta2020,pantry_count,ntaname
0,BK0101,1,Greenpoint
1,BK0102,1,Williamsburg
2,BK0103,1,South Williamsburg
3,BK0104,1,East Williamsburg
4,BK0201,0,Brooklyn Heights


## PANTRY TABLE

In [13]:
pantries_table = pantries[['id', 'program',  'geoid', 'nta2020', 'geometry']]

In [14]:
pantries_table.head()

,id,program,geoid,nta2020,geometry
0,0,CHURCH OF ST. NICHOLAS OF TOLENTINE,36005025500,BX0701,POINT (-73.905672642358 40.862216353818)
1,1,BREAD OF LIFE FOOD PANTRY,36005005002,BX0901,POINT (-73.8785391641132 40.8264247055377)
2,2,CHANCE FOR CHILDREN,36047032500,BK0901,POINT (-73.9584754469383 40.6656935813309)
3,3,HEALTH ESSENTIAL ASSOCIATION INC (BK),36047058000,BK1502,POINT (-73.955282193588 40.598606630507)
4,4,AIDS CENTER OF QUEENS COUNTY (WOODSIDE),36081026100,QN0203,POINT (-73.902187195561 40.744659030225)


## TRACT TABLE

In [15]:
tracts = pd.read_csv("data/model/tracts_table.csv")

In [17]:
tracts = tracts[['geoid','nta2020', 'geometry', 'boroname', 'TotalPop']]
tracts

,geoid,nta2020,geometry,boroname,TotalPop
0,36061000100,MN0191,MULTIPOLYGON (((-74.04387761639944 40.69018767...,Manhattan,0.0
1,36061001401,MN0302,MULTIPOLYGON (((-73.9883662631772 40.716445702...,Manhattan,3155.0
2,36061001402,MN0302,MULTIPOLYGON (((-73.98507342254645 40.71908329...,Manhattan,2932.0
3,36061001800,MN0302,MULTIPOLYGON (((-73.9898545438136 40.720520352...,Manhattan,8326.0
4,36061002201,MN0302,MULTIPOLYGON (((-73.97875234984308 40.71993370...,Manhattan,6861.0
...,...,...,...,...,...
2320,36061009903,MN0401,MULTIPOLYGON (((-73.99729876528028 40.75710704...,Manhattan,NaN
2321,36061011700,MN0401,MULTIPOLYGON (((-74.00178824088411 40.76229452...,Manhattan,3870.0
2322,36061002601,MN0303,MULTIPOLYGON (((-73.9768923033927 40.722497238...,Manhattan,4087.0
2323,36061003200,MN0303,MULTIPOLYGON (((-73.97990650235904 40.72686577...,Manhattan,7871.0


## NTA TABLE

In [18]:
nta = pd.read_csv(NTA_PATH)
SUPPLY_GAP_COL = "Supply Gap (lbs.)"
nta_table = nta[['nta2020', 'ntaname','geoid','geometry','Supply Gap (lbs.)','pantry_count',]]
nta_table[SUPPLY_GAP_COL] = nta_table[SUPPLY_GAP_COL].apply(lambda x: x.replace(',', '')).astype(float)

/var/folders/dj/dxzvsp311dl66s42c2nyqbk00000gn/T/ipykernel_78038/1678640100.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nta_table[SUPPLY_GAP_COL] = nta_table[SUPPLY_GAP_COL].apply(lambda x: x.replace(',', '')).astype(float)


In [19]:
nta_table

,nta2020,ntaname,geoid,geometry,Supply Gap (lbs.),pantry_count
0,BK0101,Greenpoint,36047056301,POINT (998241.9574088433 205056.93764567457),1.153882e+06,1
1,BK0102,Williamsburg,36047051300,POINT (995721.0164833779 199741.71509407158),1.011421e+06,1
2,BK0103,South Williamsburg,36047050900,POINT (996267.1178804908 195489.48591761288),2.090951e+06,1
3,BK0104,East Williamsburg,36047048900,POINT (1001857.1708558884 199157.09185645176),1.224484e+06,1
4,BK0201,Brooklyn Heights,36047000301,POINT (985683.5935392772 192653.28067751398),3.226514e+05,0
...,...,...,...,...,...,...
192,SI0301,Oakwood-Richmondtown,36085012805,POINT (950187.8142856951 144051.3912047569),5.297577e+05,0
193,SI0302,Great Kills-Eltingville,36085014608,POINT (941182.6447296155 139786.65114480926),1.091231e+06,1
194,SI0303,Arden Heights-Rossville,36085017007,POINT (929531.785051678 140680.3094509498),6.001747e+05,0
195,SI0304,Annadale-Huguenot-Prince's Bay-Woodrow,36085017009,POINT (929195.1521514927 131846.05201889153),6.208256e+05,2


In [24]:
EXCESS_NTA = nta_table[nta_table[SUPPLY_GAP_COL] < 0]
DEFICIT_NTA = nta_table[nta_table[SUPPLY_GAP_COL] > 0]

print(EXCESS_NTA)
print(DEFICIT_NTA)

    nta2020                                       ntaname        geoid  \
5    BK0202           Downtown Brooklyn-DUMBO-Boerum Hill  36047001502   
9    BK0302                     Bedford-Stuyvesant (East)  36047027300   
11   BK0402                               Bushwick (East)  36047039900   
12   BK0501                                 Cypress Hills  36047114202   
13   BK0502                         East New York (North)  36047113400   
14   BK0503                        East New York-New Lots  36047107800   
16   BK0505                       East New York-City Line  36047119000   
23   BK0802                         Crown Heights (North)  36047030700   
25   BK0902             Prospect Lefferts Gardens-Wingate  36047032900   
35   BK1301                             Gravesend (South)  36047030600   
39   BK1402         Flatbush (West)-Ditmas Park-Parkville  36047046000   
44   BK1601                                    Ocean Hill  36047030300   
46   BK1701                         Ea

In [23]:
excess_ids = EXCESS_NTA.nta2020
deficit_ids = DEFICIT_NTA.nta2020

print(excess_ids)
print(deficit_ids)

5      BK0202
9      BK0302
11     BK0402
12     BK0501
13     BK0502
14     BK0503
16     BK0505
23     BK0802
25     BK0902
35     BK1301
39     BK1402
44     BK1601
46     BK1701
47     BK1702
48     BK1703
49     BK1704
50     BK1801
51     BK1802
52     BK1803
54     BX0102
55     BX0201
56     BX0202
57     BX0301
59     BX0303
60     BX0401
67     BX0602
79     BX1001
88     BX1202
110    MN0802
115    MN1001
121    MN1203
126    QN0105
127    QN0201
128    QN0202
129    QN0203
131    QN0302
138    QN0504
148    QN0801
154    QN0902
155    QN0903
156    QN0904
157    QN0905
159    QN1002
165    QN1201
166    QN1202
167    QN1203
168    QN1204
169    QN1205
170    QN1206
173    QN1303
174    QN1304
175    QN1305
176    QN1306
182    SI0102
186    SI0106
Name: nta2020, dtype: object
0      BK0101
1      BK0102
2      BK0103
3      BK0104
4      BK0201
        ...  
192    SI0301
193    SI0302
194    SI0303
195    SI0304
196    SI0305
Name: nta2020, Length: 142, dtype: object


## DISTANCE

In [62]:
from haversine import haversine, Unit

# Extract lat/lon as tuples from shapely Points
excess_coords = [(p.y, p.x) for p in excess_pantries['geometry']]
deficit_coords = [(p.y, p.x) for p in deficit_pantries['geometry']]

# Initialize distance matrix
dist_matrix = np.zeros((len(excess_coords), len(deficit_coords)))

# Compute distances using haversine
for i, e_coord in enumerate(excess_coords):
    for j, d_coord in enumerate(deficit_coords):
        dist_matrix[i, j] = haversine(e_coord, d_coord, unit=Unit.MILES)

# Build DataFrame
dist_df_pantry = pd.DataFrame(
    dist_matrix,
    index=excess_pantries['id'],
    columns=deficit_pantries['id']
)

dist_df_pantry

id,0,1,2,3,5,7,8,9,10,11,...,502,503,504,505,508,509,510,511,513,514
id,,,,,,,,,,,,,,,,,,,,,
4,8.124483,5.783341,6.201623,10.467823,12.437733,5.620297,4.529374,5.094675,6.063299,3.698567,...,5.711591,11.221461,5.540709,5.611690,4.437986,6.572050,11.901670,14.474525,9.444375,1.617911
6,12.275282,9.427942,8.827752,11.022262,7.264138,11.377573,10.185154,10.693116,11.448102,9.778236,...,3.700469,15.066667,7.326476,11.261620,10.375681,11.723441,15.496740,17.245925,11.743005,4.666845
12,7.565830,6.424174,6.812850,11.447883,15.811546,3.337970,2.983074,3.390669,4.413094,0.366142,...,9.168844,10.460024,7.026577,3.585808,1.312665,5.152441,10.477360,13.476082,9.428587,5.341506
16,14.205805,11.376888,11.354230,12.897497,6.298584,13.891251,12.688975,13.165253,13.828480,12.539981,...,4.974975,16.769530,9.809622,13.738289,13.164488,14.016757,17.999945,19.522986,14.060632,7.330217
19,7.213798,4.506699,8.125161,12.141911,12.416536,6.013764,4.811701,5.296196,6.027550,5.323343,...,3.895053,10.219024,7.251961,5.868298,6.223353,6.323140,14.002957,16.536076,11.405022,1.320888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,12.747479,9.899016,9.890040,11.904442,7.053819,12.175896,10.975654,11.464230,12.164470,10.764458,...,3.733946,15.433451,8.376287,12.036354,11.395548,12.389642,16.559184,18.256485,12.758385,5.562527
496,13.498711,11.009863,3.653903,5.887440,8.349878,10.716814,9.759066,10.327752,11.354381,7.626203,...,7.975518,16.588013,2.095761,10.789597,7.683442,11.917453,10.241289,11.840654,6.341123,5.284899
506,13.190393,10.887561,2.239623,5.431842,9.755161,10.001620,9.149312,9.707626,10.764988,6.654785,...,8.742891,16.289330,0.716176,10.121542,6.560246,11.384106,8.898103,10.701314,5.214303,5.451073
